In [1]:
%%capture output
!pip install --upgrade pip
!pip install --upgrade pandas
!pip install tables   
# necessary for pd.read_hdf()

!pip install ipywidgets
!pip install --upgrade jupyter
!pip install IProgress
!pip install catboost
!pip install shap
!pip install anndata

In [4]:
import os
import numpy as np
import pandas as pd
import anndata as ad

In [3]:
%matplotlib inline
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

## data load

In [4]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/open-problems-multimodal-3rd-solution/'

# model_path_for_now = '/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/'

raw_path =  lrz_path + 'input/raw/'  # '../../../input/raw/'

cite_target_path = lrz_path + 'input/target/cite/'   # '../../../input/target/cite/'
cite_feature_path = lrz_path + 'input/features/cite/'   # '../../../input/features/cite/'
cite_mlp_path = lrz_path + 'model/cite/mlp/'   # '../../../model/cite/mlp/'   # '../../../model/cite/mlp/'
cite_cb_path = lrz_path + 'model/cite/cb/'   # '../../../model/cite/cb/'

# multi_target_path = lrz_path + 'input/target/multi/'   # '../../../input/target/multi/'
# multi_feature_path = lrz_path + 'input/features/multi/'   # '../../../input/features/multi/'
# multi_mlp_path = lrz_path + 'model/multi/mlp/'   # '../../../model/multi/mlp/'
# multi_cb_path = lrz_path + 'model/multi/cb/'   # '../../../model/multi/cb/'

index_path = lrz_path + 'input/preprocess/cite/'

output_path = lrz_path + 'output/'   # '../../../output/'

## Cite

In [5]:
# get model name
#mlp_model_path = os.listdir(cite_mlp_path)

markdown

In [ ]:
# check model names and lists/dict/...

In [2]:
mlp_model_name = [
    'corr_add_con_imp',
    'corr_last_v3', 
    'corr_c_add_w2v_v1_mish_flg',
    'corr_c_add_w2v_v1_flg',
    'corr_c_add_84_v1',
    'corr_c_add_120_v1',
    'corr_w2v_cell_flg',
    'corr_best_cell_120',
    'corr_cluster_cell',
    'corr_w2v_128',
    'corr_imp_w2v_128',
    'corr_snorm',
    'corr_best_128',
    'corr_best_64',
    'corr_cluster_128',
    'corr_cluster_64',
    'corr_svd_128',
    'corr_svd_64',
             ]

18

In [7]:
model_name_list = []

for i in mlp_model_name:
    for num, j in enumerate(os.listdir(cite_mlp_path)):
        if i in j:
            model_name_list.append(j)

len(model_name_list)
model_name_list   # TODO why not listdir(cite_mlp_path) directly? always 18 models... does mlp_model_name list come from somewhere else?

['cite_mlp_corr_add_con_imp_flg_donor_val_50',
 'cite_mlp_corr_last_v3_flg_donor_val_55',
 'cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66',
 'cite_mlp_corr_c_add_w2v_v1_flg_donor_val_66',
 'cite_mlp_corr_c_add_84_v1_flg_donor_val_47',
 'cite_mlp_corr_c_add_120_v1_flg_donor_val_63',
 'cite_mlp_corr_w2v_cell_flg_donor_val_51',
 'cite_mlp_corr_best_cell_120_flg_donor_val_51',
 'cite_mlp_corr_cluster_cell_flg_donor_val_64',
 'cite_mlp_corr_w2v_128_flg_donor_val_42',
 'cite_mlp_corr_imp_w2v_128_flg_donor_val_38',
 'cite_mlp_corr_snorm_flg_donor_val_39',
 'cite_mlp_corr_best_128_flg_donor_val_45',
 'cite_mlp_corr_best_64_flg_donor_val_50',
 'cite_mlp_corr_cluster_128_flg_donor_val_51',
 'cite_mlp_corr_cluster_64_flg_donor_val_57',
 'cite_mlp_corr_svd_128_flg_donor_val_30',
 'cite_mlp_corr_svd_64_flg_donor_val_38']

In [6]:
model_name_list = ['cite_mlp_corr_add_con_imp_flg_donor_val_50',
 'cite_mlp_corr_last_v3_flg_donor_val_55',
 'cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66',
 'cite_mlp_corr_c_add_w2v_v1_flg_donor_val_66',
 'cite_mlp_corr_c_add_84_v1_flg_donor_val_47',
 'cite_mlp_corr_c_add_120_v1_flg_donor_val_63',
 'cite_mlp_corr_w2v_cell_flg_donor_val_51',
 'cite_mlp_corr_best_cell_120_flg_donor_val_51',
 'cite_mlp_corr_cluster_cell_flg_donor_val_64',
 'cite_mlp_corr_w2v_128_flg_donor_val_42',
 'cite_mlp_corr_imp_w2v_128_flg_donor_val_38',
 'cite_mlp_corr_snorm_flg_donor_val_39',
 'cite_mlp_corr_best_128_flg_donor_val_45',
 'cite_mlp_corr_best_64_flg_donor_val_50',
 'cite_mlp_corr_cluster_128_flg_donor_val_51',
 'cite_mlp_corr_cluster_64_flg_donor_val_57',
 'cite_mlp_corr_svd_128_flg_donor_val_30',
 'cite_mlp_corr_svd_64_flg_donor_val_38']

In [17]:
weight = [1, 0.3, 1, 1, 1, 1, 1, 1, 1, 0.8, 0.8, 0.8, 0.8, 0.5, 0.5, 0.5, 1, 1, 2, 2]
weight_sum = np.array(weight).sum()    # == 19

# model_feat_dict = {model_name_list[0]:['X_test_add_con_imp.pickle', 1],
#                    model_name_list[1]:['X_test_last_v3.pickle', 0.3],
#                    model_name_list[2]:['X_test_c_add_w2v_v1.pickle', 1],
#                    model_name_list[3]:['X_test_c_add_w2v_v1.pickle', 1],
#                    model_name_list[4]:['X_test_c_add_84_v1.pickle', 1],
#                    model_name_list[5]:['X_test_c_add_v1.pickle', 1],
                   
#                    model_name_list[6]:['X_test_feature_w2v_cell.pickle', 1],
#                    model_name_list[7]:['X_test_best_cell_128_120.pickle', 1],
#                    model_name_list[8]:['X_test_cluster_cell_128.pickle', 1],
                   
#                    model_name_list[9]:['X_test_feature_w2v.pickle', 0.8],
#                    model_name_list[10]:['X_test_feature_imp_w2v.pickle',0.8],
#                    model_name_list[11]:['X_test_feature_snorm.pickle', 0.8],
#                    model_name_list[12]:['X_test_best_128.pickle', 0.8],
#                    model_name_list[13]:['X_test_best_64.pickle', 0.5],
#                    model_name_list[14]:['X_test_cluster_128.pickle', 0.5],
#                    model_name_list[15]:['X_test_cluster_64.pickle', 0.5],
#                    model_name_list[16]:['X_test_svd_128.pickle', 1],
#                    model_name_list[17]:['X_test_svd_64.pickle', 1],
                   
#                    'best_128':['X_test_best_128.pickle', 2],
#                    'best_64':['X_test_best_64.pickle', 2],
#                   }

In [15]:
# create list of file names to have model_feat_dict in a cleaner way... but is it still readable? TODO
file_names = ['X_test_add_con_imp.pickle',
 'X_test_last_v3.pickle',
 'X_test_c_add_w2v_v1.pickle',
 'X_test_c_add_w2v_v1.pickle',
 'X_test_c_add_84_v1.pickle',
 'X_test_c_add_v1.pickle',
 'X_test_feature_w2v_cell.pickle',
 'X_test_best_cell_128_120.pickle',
 'X_test_cluster_cell_128.pickle',
 'X_test_feature_w2v.pickle',
 'X_test_feature_imp_w2v.pickle',
 'X_test_feature_snorm.pickle',
 'X_test_best_128.pickle',
 'X_test_best_64.pickle',
 'X_test_cluster_128.pickle',
 'X_test_cluster_64.pickle',
 'X_test_svd_128.pickle',
 'X_test_svd_64.pickle',
 'X_test_best_128.pickle',
 'X_test_best_64.pickle']

model_feat_dict = {name: [file_name, weight] for name, weight, file_name in zip(model_name_list+['best_128', 'best_64'], weight, file_names)}


### cite model

In [9]:
def std(x):
    x = np.array(x)
    return (x - x.mean(1).reshape(-1, 1)) / x.std(1).reshape(-1, 1)

In [10]:
# class CiteDataset(Dataset):
    
#     def __init__(self, feature, target):
        
#         self.feature = feature
#         self.target = target
        
#     def __len__(self):
#         return len(self.feature)
    
#     def __getitem__(self, index):
                
#         d = {
#             "X": self.feature[index],
#             "y" : self.target[index],
#         }
#         return d

In [11]:
class CiteDataset_test(Dataset):
    
    def __init__(self, feature):
        self.feature = feature
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index]
        }
        return d

In [12]:
# def partial_correlation_score_torch_faster(y_true, y_pred):
#     """Compute the correlation between each rows of the y_true and y_pred tensors.
#     Compatible with backpropagation.
#     """
#     y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
#     y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
#     cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
#     var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
#     var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
#     return cov_tp/torch.sqrt(var_t*var_p)

# def correl_loss(pred, tgt):
#     """Loss for directly optimizing the correlation.
#     """
#     return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))

In [13]:
class CiteModel(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.ReLU(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.ReLU(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.ReLU(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
        
        from_numpy = False
        
      ##
        if isinstance(X, np.ndarray):
            X = torch.from_numpy(X)
            from_numpy = True
        X = X.to(device)  # Move the input to the appropriate device if necessary
        ##
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        if from_numpy:
            out = out.cpu().detach().numpy()
            
        return out

In [14]:
class CiteModel_mish(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel_mish, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.Mish(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.Mish(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.Mish(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
    
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [15]:
# def train_loop(model, optimizer, loader, epoch):
    
#     losses, lrs = [], []
#     model.train()
#     optimizer.zero_grad()
#     #loss_fn = nn.MSELoss()
    
#     with tqdm(total=len(loader),unit="batch") as pbar:
#         pbar.set_description(f"Epoch{epoch}")
        
#         for d in loader:
#             X = d['X'].to(device)
#             y = d['y'].to(device)
            
#             logits = model(X)
#             loss = correl_loss(logits, y)
#             #loss = torch.sqrt(loss_fn(logits, y))
        
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             pbar.set_postfix({"loss":loss.item()})
#             pbar.update(1)

#     return model

In [16]:
# def valid_loop(model, loader, y_val):
    
#     model.eval()
#     partial_correlation_scores = []
#     oof_pred = []
    
#     for d in loader:
#         with torch.no_grad():
#             val_X = d['X'].to(device).float()
#             val_y = d['y'].to(device)
#             logits = model(val_X)
#             oof_pred.append(logits)
    
#     #print(torch.cat(oof_pred).shape, torch.cat(oof_pred).detach().cpu().numpy().shape)
#     cor = partial_correlation_score_torch_faster(torch.tensor(y_val).to(device), torch.cat(oof_pred))
#     cor = cor.mean().item()
#     logits = torch.cat(oof_pred).detach().cpu().numpy()
    
#     return logits, cor

In [17]:
def test_loop(model, loader):
    
    model.eval()
    predicts=[]

    for d in tqdm(loader):
        with torch.no_grad():
            X = d['X'].to(device)
            logits = model(X)
            predicts.append(logits.detach().cpu().numpy())
            
    return np.concatenate(predicts)

### ensemble prediction: use all models, add weighted predictions

In [18]:
pred = np.zeros([48203, 140])

for num, i in enumerate(model_feat_dict.keys()):
    
    print(i)
    
    if 'mlp' in i:

        try:
            test_file = model_feat_dict[i][0]
            test_weight = model_feat_dict[i][1]
            X_test = pd.read_pickle(cite_feature_path  + test_file)   
            # print(cite_feature_path  + test_file)
            X_test = np.array(X_test)
            feature_dims = X_test.shape[1]

            test_ds = CiteDataset_test(X_test)
            test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                         shuffle=False, drop_last=False, num_workers=4)

            if 'mish' in i:
                model = CiteModel_mish(feature_dims)
            else:
                model = CiteModel(feature_dims)

            model = model.to(device)
            model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

            result = test_loop(model, test_dataloader).astype(np.float32)
            result = std(result) * test_weight / weight_sum
            pred += result

            torch.cuda.empty_cache()

        except Exception as e: 
            print(i)
            print(e)             # TODOOOOOOOOOOOOOO
        
    else:
        test_file = model_feat_dict[i][0]
        test_weight = model_feat_dict[i][1]
        X_test = pd.read_pickle(cite_feature_path  + test_file)
        
        cb_pred = np.zeros([48203, 140])
        
        for t in tqdm(range(140)): 
            cb_model_path = [j for j in os.listdir(cite_cb_path) if f'cb_{t}_{i}' in j][0]
            cb = pickle.load(open(cite_cb_path + cb_model_path, 'rb'))
            cb_pred[:,t] = cb.predict(X_test)
            
        cb_pred = cb_pred.astype(np.float32)
        pred += std(cb_pred) * test_weight / weight_sum
        
        del cb_pred

cite_mlp_corr_add_con_imp_flg_donor_val_50


  0%|          | 0/377 [00:00<?, ?it/s]

cite_mlp_corr_last_v3_flg_donor_val_55


  0%|          | 0/377 [00:00<?, ?it/s]

cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66


...

best_128


  0%|          | 0/140 [00:00<?, ?it/s]

TBB Warning: The number of workers is currently limited to 9. The request for 79 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



best_64


  0%|          | 0/140 [00:00<?, ?it/s]

In [19]:
pd.DataFrame(pred.round(6))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-0.325767,-0.487738,-0.351139,1.035394,0.867302,2.263471,3.314403,-0.490992,-0.422089,-0.312560,-0.516754,-0.479289,-0.392685,-0.437807,2.248444,-0.356812,1.858463,1.438221,0.311931,-0.459447,-0.390223,0.357044,-0.487529,-0.386469,2.933681,-0.493526,-0.496832,-0.490686,-0.519472,-0.486443,-0.499483,-0.456832,-0.462952,-0.465901,-0.445472,-0.486322,-0.406220,3.706107,-0.426987,-0.426969,-0.478369,-0.473534,-0.493516,-0.463637,-0.521282,-0.477247,-0.308790,-0.352572,2.829829,-0.491238,-0.471341,-0.467674,-0.427709,-0.501900,-0.094166,0.147856,-0.470777,1.119057,-0.465110,-0.468767,-0.465743,-0.437902,-0.412044,-0.464638,-0.483274,-0.425702,-0.353300,-0.406926,0.377374,-0.491446,-0.499591,-0.521969,-0.460512,3.233928,-0.502028,1.961053,-0.459225,0.279651,-0.455238,-0.469370,0.169536,-0.508457,-0.244526,-0.434981,-0.501910,-0.498316,-0.441767,-0.476339,-0.357719,-0.479157,-0.351639,-0.498732,-0.471366,-0.378029,0.257164,-0.401403,-0.442544,1.844755,-0.465230,1.572834,0.957418,-0.488895,1.072358,-0.422123,0.028856,-0.414253,1.396735,-0.268038,2.183714,-0.345104,-0.337413,0.395187,-0.390905,-0.465734,-0.479009,0.327426,-0.144475,-0.422026,-0.452289,2.622928,-0.450933,0.406673,-0.475251,-0.482181,-0.423303,-0.416871,-0.487996,-0.307121,-0.448958,-0.488162,-0.514007,1.841981,-0.389785,-0.518213,-0.500604,-0.341130,-0.016153,-0.317298,0.038913,0.358591
1,-0.293307,-0.470185,-0.351417,1.042340,0.918223,2.295353,3.557569,-0.485068,-0.415240,-0.313825,-0.507285,-0.467309,-0.390135,-0.437772,2.081140,-0.379274,1.817812,1.527190,0.218958,-0.443928,-0.396843,0.337870,-0.473521,-0.378367,2.997932,-0.475288,-0.487909,-0.482030,-0.519096,-0.478371,-0.486569,-0.440771,-0.453228,-0.449638,-0.432872,-0.484526,-0.406047,3.649258,-0.414698,-0.415922,-0.476712,-0.462437,-0.486182,-0.507216,-0.513429,-0.466063,-0.308042,-0.362187,2.793647,-0.483065,-0.458920,-0.464745,-0.472144,-0.494432,-0.116090,0.168782,-0.459241,0.760353,-0.473271,-0.456876,-0.456543,-0.430272,-0.406301,-0.449704,-0.480652,-0.419016,-0.357765,-0.404320,0.361015,-0.486164,-0.483510,-0.508212,-0.455934,3.269518,-0.497218,1.836228,-0.448956,0.262899,-0.452102,-0.455511,0.015679,-0.488521,-0.228284,-0.424335,-0.491802,-0.481530,-0.433237,-0.465633,-0.349264,-0.457304,-0.351280,-0.485360,-0.462076,-0.375112,0.195090,-0.387814,-0.433895,1.852686,-0.458732,1.629971,0.989916,-0.481465,1.042104,-0.415026,-0.026993,-0.408080,1.243898,-0.315948,2.307294,-0.338717,-0.338424,0.279150,-0.370051,-0.454253,-0.466977,0.202053,-0.207233,-0.402973,-0.442968,2.643998,-0.447381,0.725545,-0.461757,-0.474494,-0.407510,-0.405384,-0.473229,-0.275020,-0.438769,-0.481086,-0.503055,1.824850,-0.384038,-0.505780,-0.490800,-0.358872,-0.072174,-0.322904,0.059740,0.321151
2,-0.337575,-0.449081,-0.328770,1.541549,1.271137,1.486816,4.438023,-0.463581,-0.391279,-0.423727,-0.490480,-0.432064,-0.413590,-0.419570,3.426018,-0.273052,1.050578,1.232067,-0.147889,-0.459557,-0.391296,0.349071,-0.452589,-0.377862,2.946249,-0.457357,-0.465505,-0.456003,-0.552835,-0.449007,-0.464044,-0.396615,-0.427664,-0.421790,-0.406941,-0.461765,-0.376789,3.081319,-0.390267,-0.414918,-0.445042,-0.445852,-0.436071,-0.241519,-0.477286,-0.428363,-0.259145,-0.387482,0.834384,-0.462517,-0.428094,-0.450838,-0.397854,-0.472625,-0.105831,0.155859,-0.442649,0.735279,-0.463939,-0.440942,-0.214496,-0.391008,-0.372151,-0.421942,-0.428422,-0.395630,-0.315540,-0.405780,0.398103,-0.453094,-0.445499,-0.482222,-0.441426,1.487004,-0.439823,1.227368,-0.422020,0.056478,-0.423029,-0.428986,-0.157021,-0.457496,-0.1489

### pred loop function (only for single models, not ensemble)

In [ ]:
def get_pred(model_name, X_test=None):     # , rows, cols): 

    '''if test file from model_feat_dict: pass X_test=None
    if other test file, e.g. private test data, pass it as param '''


    # pred = np.zeros([rows, cols])
    if X_test is None:
        test_file = model_feat_dict[model_name][0]
        X_test = np.array(pd.read_pickle(cite_feature_path + test_file))
    
    feature_dims = X_test.shape[1]

    test_ds = CiteDataset_test(X_test)
    test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                shuffle=False, drop_last=False, num_workers=4)

    if 'mish' in model_name:
        model = CiteModel_mish(feature_dims)
    else:
        model = CiteModel(feature_dims)
        
    model = model.to(device)
    model.load_state_dict(torch.load(f'{cite_mlp_path}/{model_name}'))

    pred = test_loop(model, test_dataloader).astype(np.float32)
   
    torch.cuda.empty_cache()
            
    return pd.DataFrame(pred)

use for model 16:

In [21]:
get_pred('cite_mlp_corr_svd_128_flg_donor_val_30')

# # model #16: cite_mlp_corr_svd_128_flg_donor_val_30
# pred_16 = np.zeros([48203, 140])

# i = 'cite_mlp_corr_svd_128_flg_donor_val_30'
        
# test_file = model_feat_dict[i][0]
# # test_weight = model_feat_dict[i][1]
# X_test = pd.read_pickle(cite_feature_path  + test_file)
# X_test = np.array(X_test)
# feature_dims = X_test.shape[1]

# test_ds = CiteDataset_test(X_test)
# test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
#                               shuffle=False, drop_last=False, num_workers=4)

# if 'mish' in i:
#     model = CiteModel_mish(feature_dims)
# else:
#     model = CiteModel(feature_dims)
    
# model = model.to(device)
# model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

# result = test_loop(model, test_dataloader).astype(np.float32)
# pred_16 += result

# torch.cuda.empty_cache()
        
# pd.DataFrame(pred_16)   # double check train_cite_targets.h5  -> omnipath

  0%|          | 0/377 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-1.066433,-1.500550,-1.354774,1.106190,0.869803,3.481040,5.067837,-1.570396,-1.444474,-1.320852,-1.612052,-1.534098,-1.428519,-1.496802,3.095008,-1.384286,2.648872,1.938160,-0.093022,-1.635664,-1.428120,-0.072564,-1.464633,-1.333522,4.528148,-1.577123,-1.570215,-1.564498,-1.633450,-1.551511,-1.606082,-1.488960,-1.542044,-1.542291,-1.515924,-1.584970,-1.483199,6.078822,-1.523234,-1.523154,-1.628240,-1.568818,-1.554284,-1.576391,-1.607019,-1.563618,-1.300677,-1.282699,4.529342,-1.497951,-1.504462,-1.616715,-1.604992,-1.590265,-0.913128,-0.373880,-1.571010,1.285669,-1.561719,-1.562544,-1.531397,-1.546362,-1.493735,-1.614805,-1.581547,-1.436415,-1.352806,-1.433268,0.004768,-1.507400,-1.582382,-1.650515,-1.553527,5.308838,-1.614917,2.937888,-1.565187,-0.132013,-1.512913,-1.525913,-0.179205,-1.617044,-1.141805,-1.497630,-1.589817,-1.619898,-1.517949,-1.527373,-1.324593,-1.536239,-1.349147,-1.607809,-1.596236,-1.449520,-0.069496,-1.395786,-1.459246,2.709844,-1.504532,2.281004,1.127239,-1.582563,1.390550,-1.485366,-0.814275,-1.483867,1.818746,-1.166362,3.551317,-1.324322,-1.333104,-0.007823,-1.370018,-1.791438,-1.608289,-0.100262,-0.915906,-1.476855,-1.569585,4.021511,-1.475899,0.135406,-1.601376,-1.499214,-1.508077,-1.504729,-1.617144,-1.285858,-1.497512,-1.602304,-1.585749,2.483784,-1.404202,-1.656756,-1.628249,-1.330432,-1.008113,-1.255932,-0.622936,-0.130905
1,-1.067110,-1.506955,-1.399289,1.075526,0.997081,3.506622,5.481764,-1.564466,-1.460833,-1.301538,-1.630497,-1.572419,-1.490347,-1.531179,3.168335,-1.427267,2.470253,2.130677,-0.441134,-1.637434,-1.472109,-0.058242,-1.472134,-1.346540,4.738451,-1.580705,-1.617835,-1.605957,-1.660571,-1.576319,-1.645333,-1.515913,-1.583853,-1.545092,-1.542181,-1.602096,-1.507277,6.134613,-1.544582,-1.598276,-1.658122,-1.569817,-1.591216,-1.618025,-1.622394,-1.573258,-1.365324,-1.374641,4.661467,-1.514943,-1.525232,-1.650555,-1.612984,-1.614313,-1.050634,-0.368892,-1.596666,0.777737,-1.589263,-1.581116,-1.578929,-1.556408,-1.516185,-1.611275,-1.630935,-1.470598,-1.411295,-1.485477,-0.118224,-1.505585,-1.588891,-1.651967,-1.588987,5.179552,-1.639723,2.675957,-1.584337,-0.139389,-1.552935,-1.552968,-0.562470,-1.613981,-1.148527,-1.525604,-1.601094,-1.632247,-1.561806,-1.547703,-1.396585,-1.549064,-1.390192,-1.631668,-1.603218,-1.467851,-0.253814,-1.409238,-1.494805,2.745513,-1.549431,2.249057,0.980224,-1.594142,1.342944,-1.517599,-0.857558,-1.512639,1.533982,-1.339978,3.737477,-1.391075,-1.371431,-0.343284,-1.380614,-1.574345,-1.623899,-0.251694,-1.224326,-1.529244,-1.581402,4.151693,-1.492653,0.829439,-1.611794,-1.553943,-1.531965,-1.514008,-1.643932,-1.266751,-1.511899,-1.586327,-1.620505,2.777343,-1.404117,-1.657573,-1.687846,-1.422966,-0.780818,-1.299758,-0.628505,-0.121618
2,-1.010656,-1.271471,-1.207577,2.074188,1.614956,2.007590,6.589465,-1.374983,-1.260541,-1.450437,-1.464440,-1.332187,-1.378543,-1.357404,4.944262,-1.135714,0.924946,1.518677,-0.864071,-1.520523,-1.308277,0.001427,-1.352170,-1.241024,4.492778,-1.413115,-1.387910,-1.474926,-1.551057,-1.376167,-1.415526,-1.233463,-1.391570,-1.319213,-1.332545,-1.431528,-1.287814,4.748064,-1.345603,-1.431643,-1.396456,-1.397756,-1.360939,-1.085002,-1.420398,-1.334369,-1.064852,-1.288315,1.083776,-1.353123,-1.357820,-1.394112,-1.357486,-1.375918,-0.872777,-0.296593,-1.410997,0.365104,-1.434169,-1.399326,-0.980195,-1.282730,-1.286808,-1.391266,-1.348446,-1.279712,-1.213837,-1.296896,0.074692,-1.323143,-1.357035,-1.430854,-1.481354,2.077808,-1.360545,1.681203,-1.313775,-0.382094,-1.374308,-1.361059,-0.

### prediction with private test input -> should get private test target

In [22]:
private_test_input = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_input.h5ad')
private_test_input

AnnData object with n_obs × n_vars = 26867 × 22085
    obs: 'kaggle_dataset', 'day', 'donor', 'cell_type'

In [23]:
private_test_target = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_target.h5ad')
private_test_target

AnnData object with n_obs × n_vars = 26867 × 140
    obs: 'kaggle_dataset', 'day', 'donor', 'cell_type'

In [134]:
# private_test_input.to_df()

gene_ids,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,ENSG00000115977_AAK1,ENSG00000087884_AAMDC,ENSG00000127837_AAMP,ENSG00000129673_AANAT,ENSG00000131043_AAR2,ENSG00000205002_AARD,ENSG00000090861_AARS,ENSG00000124608_AARS2,ENSG00000266967_AARSD1,ENSG00000157426_AASDH,ENSG00000149313_AASDHPPT,ENSG00000008311_AASS,ENSG00000215458_AATBC,ENSG00000275700_AATF,ENSG00000181409_AATK,ENSG00000281376_ABALON,ENSG00000183044_ABAT,ENSG00000165029_ABCA1,ENSG00000154263_ABCA10,ENSG00000251595_ABCA11P,ENSG00000179869_ABCA13,ENSG00000238098_ABCA17P,ENSG00000107331_ABCA2,ENSG00000167972_ABCA3,ENSG00000154265_ABCA5,ENSG00000154262_ABCA6,ENSG00000064687_ABCA7,ENSG00000085563_ABCB1,ENSG00000135776_ABCB10,ENSG00000073734_ABCB11,ENSG00000005471_ABCB4,ENSG00000115657_ABCB6,ENSG00000131269_ABCB7,ENSG00000197150_ABCB8,ENSG00000150967_ABCB9,ENSG00000103222_ABCC1,ENSG00000124574_ABCC10,ENSG00000243064_ABCC13,ENSG00000023839_ABCC2,ENSG00000108846_ABCC3,ENSG00000125257_ABCC4,ENSG00000114770_ABCC5,ENSG00000091262_ABCC6,ENSG00000101986_ABCD1,ENSG00000117528_ABCD3,ENSG00000119688_ABCD4,ENSG00000164163_ABCE1,ENSG00000204574_ABCF1,ENSG00000033050_ABCF2,ENSG00000161204_ABCF3,ENSG00000160179_ABCG1,ENSG00000118777_ABCG2,ENSG00000143921_ABCG8,ENSG00000143994_ABHD1,ENSG00000144827_ABHD10,ENSG00000106077_ABHD11,ENSG00000225969_ABHD11-AS1,ENSG00000100997_ABHD12,ENSG00000131969_ABHD12B,ENSG00000139826_ABHD13,ENSG00000248487_ABHD14A,ENSG00000114786_ABHD14A-ACY1,ENSG00000114779_ABHD14B,ENSG00000168792_ABHD15,ENSG00000264031_ABHD15-AS1,ENSG00000204427_ABHD16A,ENSG00000129968_ABHD17A,ENSG00000250536_ABHD17AP3,ENSG00000107362_ABHD17B,ENSG00000136379_ABHD17C,ENSG00000164074_ABHD18,ENSG00000140526_ABHD2,ENSG00000158201_ABHD3,ENSG00000100439_ABHD4,ENSG00000011198_ABHD5,ENSG00000163686_ABHD6,ENSG00000127220_ABHD8,ENSG00000136754_ABI1,ENSG00000138443_ABI2,ENSG00000108798_ABI3,ENSG00000154175_ABI3BP,ENSG00000097007_ABL1,ENSG00000143322_ABL2,ENSG00000099204_ABLIM1,ENSG00000173210_ABLIM3,ENSG00000175164_ABO,ENSG00000159842_ABR,ENSG00000146386_ABRACL,ENSG00000163322_ABRAXAS1,ENSG00000165660_ABRAXAS2,ENSG00000146109_ABT1,ENSG00000114626_ABTB1,ENSG00000166016_ABTB2,ENSG00000260246_AC000032.1,ENSG00000185065_AC000068.1,ENSG00000273212_AC000068.2,ENSG00000273300_AC000068.3,ENSG00000235776_AC000089.1,ENSG00000243107_AC000120.1,ENSG00000279265_AC000123.2,ENSG00000280347_AC000123.3,ENSG00000278727_AC000403.1,ENSG00000274898_AC001226.1,ENSG00000283208_AC001226.2,ENSG00000272330_AC002044.1,ENSG00000279591_AC002044.2,ENSG00000213683_AC002056.1,ENSG00000273216_AC002059.1,ENSG00000248636_AC002070.1,ENSG00000237729_AC002075.2,ENSG00000265975_AC002091.1,ENSG00000266389_AC002091.2,ENSG00000265618_AC002094.2,ENSG00000277450_AC002094.4,ENSG00000267698_AC002116.2,ENSG00000268947_AC002128.1,ENSG00000235560_AC002310.1,ENSG00000239791_AC002310.2,ENSG00000278922_AC002310.6,ENSG00000261433_AC002347.1,ENSG00000278993_AC002350.1,ENSG00000232392_AC002366.1,ENSG00000283638_AC002407.1,ENSG00000237819_AC002454.1,ENSG00000223969_AC002456.1,ENSG00000260188_AC002464.1,ENSG00000241764_AC002467.1,ENSG00000272829_AC002470.1,ENSG00000284060_AC002472.2,ENSG00000235945_AC002543.1,ENSG00000261312_AC002550.1,ENSG00000276571_AC002550.2,ENSG00000227782_AC002553.1,ENSG00000275413_AC002553.2,ENSG00000239291_AC002558.1,ENSG00000262265_AC002558.3,ENSG00000260793_AC003102.1,ENSG00000279159_AC003681.1,ENSG00000263280_AC003965.1,ENSG00000279377_AC003973.3,ENSG00000228113_AC003991.1,ENSG00000254003_AC003991.2,ENSG00000237903_AC004000.1,ENSG00000262482_AC004034.1,ENSG00000196656_AC004057.1,ENSG00000273447_AC004067.1,ENSG00000251259_AC004069.1,ENSG00000276449_AC004076.2,ENSG00000270182_AC004080.4,ENSG00000271133_AC004130.1,ENSG00000263272_AC004148.2,ENSG00000267666_AC004156.1,ENSG00000260664_AC004158.1,ENSG00000267457_AC004223.2,ENSG00000273687_AC004223.3,ENS

In [24]:
# should be non-raw df (non-int)
# private_test_target_raw = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_target_raw.h5ad')
# private_test_target_raw

AnnData object with n_obs × n_vars = 26867 × 140
    obs: 'kaggle_dataset', 'day', 'donor', 'cell_type'

In [117]:
with open('private_X_test_svd.pkl', 'rb') as f:  # private_X_test_svd

    private_X_test_svd = pickle.load(f)
private_X_test_svd.shape

(26867, 212)

In [118]:
# with open('private_X_test_svd_from_raw.pkl', 'rb') as f:  # private_X_test_svd

#     private_X_test_svd_from_raw = pickle.load(f)
# private_X_test_svd_from_raw.shape

(26867, 212)

In [121]:
# get model #16 prediction on private test data

pred_private = get_pred('cite_mlp_corr_svd_128_flg_donor_val_30', private_X_test_svd)
pred_private

# # model #16: cite_mlp_corr_svd_128_flg_donor_val_30
# pred_16 = np.zeros([26867, 140])

# i = 'cite_mlp_corr_svd_128_flg_donor_val_30'
        
# # test_file = model_feat_dict[i][0]
# # test_weight = model_feat_dict[i][1]
# X_test = private_X_test_svd
# X_test = np.array(X_test)
# feature_dims = X_test.shape[1]

# test_ds = CiteDataset_test(X_test)
# test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
#                               shuffle=False, drop_last=False, num_workers=4)

# if 'mish' in i:
#     model = CiteModel_mish(feature_dims)
# else:
#     model = CiteModel(feature_dims)
    
# model = model.to(device)
# model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

# result = test_loop(model, test_dataloader).astype(np.float32)
# pred_16 += result

# torch.cuda.empty_cache()
        
# pd.DataFrame(pred_16)

  0%|          | 0/210 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-0.373409,-1.285452,-1.181257,0.866390,0.147068,1.610274,5.513695,-1.326972,-1.120104,-0.849544,-1.430125,-1.486018,-1.378166,-1.343360,2.520270,0.189322,0.162977,-1.085166,-1.083710,-1.661598,-1.197479,-1.001668,-1.264128,-1.176874,2.679246,-1.254575,-1.424422,-1.443570,-0.993520,-1.317004,-1.420167,-1.264446,-1.431532,-1.343266,-1.369745,-1.370425,-1.257189,2.551772,-1.254103,-1.393616,-1.384290,-1.347462,-1.278827,2.830252,-1.482109,-1.379418,-1.201816,-1.184600,1.441928,-1.385697,-1.283115,-1.325261,-0.906209,-1.495993,-0.954786,-0.975794,-1.322119,-0.762074,-1.495031,-1.133250,-1.306756,-1.171276,-1.266184,-1.318471,-0.976235,-1.243684,-1.179369,-1.182111,-0.232999,-1.121674,-1.307642,-1.358610,-1.438907,3.142342,-1.414697,1.047477,-1.269252,0.036738,-1.469478,-1.424824,0.210739,-1.412479,-0.958676,-1.333682,-1.472088,-1.438812,-1.125612,-1.365534,0.425422,-0.619397,-1.122050,-1.388966,-1.222269,-1.240194,-0.861236,-1.097354,-1.274096,1.725534,-1.389655,1.128693,0.053076,-1.416595,1.514375,-1.323590,-0.917603,-1.216529,1.081372,-1.583458,0.119425,0.640462,-0.957383,-0.422741,-0.967377,-1.393290,-1.186884,-1.327693,0.272155,-1.336972,-1.274318,2.925074,-1.253881,1.465566,-1.384402,-1.498458,-1.216529,-1.244578,-1.457050,-1.012439,-1.347274,-1.076095,-1.457051,2.210021,-1.219851,-1.357595,-1.051980,-1.163242,1.359299,-0.878143,-0.547154,0.439274
1,-0.972495,-1.184871,-1.100264,1.110875,1.479862,1.612517,2.388191,-1.161401,-1.084497,-0.937716,-1.308460,-1.320031,-1.274108,-1.216669,2.997455,-0.675659,0.505796,-0.076950,0.563774,-0.978574,-1.051314,-0.072966,-1.135781,-1.087284,3.529760,-1.250505,-1.296861,-1.313284,-1.232247,-1.194289,-1.274393,-1.183101,-1.299031,-1.270256,-1.206910,-1.288069,-1.186185,4.750784,-1.198896,-1.282160,-1.316083,-1.306794,-1.234292,-0.682397,-1.306165,-1.251352,-1.166209,-1.109901,2.581428,-1.289152,-1.192521,-1.195606,-0.522530,-1.317116,-0.816040,-0.326051,-1.172553,0.298161,-1.322378,-1.268417,-1.271458,-1.149538,-1.204418,-1.210375,-1.327354,-1.182121,-1.225290,-0.900365,-0.257900,-1.310845,-1.141683,-1.216353,-1.360508,3.018631,-1.275923,2.880551,-1.195193,-0.356693,-1.242887,-1.281857,0.603517,-1.199618,-0.769740,-1.283595,-1.304527,-1.369265,-1.198345,-1.248291,-0.862196,-0.632556,-1.088296,-1.336518,-1.146470,-1.145936,0.264749,-1.128373,-1.175346,1.600897,-1.218518,1.519524,0.102490,-1.298417,1.333938,-1.200821,-0.245253,-1.180777,1.197729,-1.351019,0.660956,-1.015569,-1.056139,-0.223279,-0.972506,-0.933463,-1.204788,0.581791,-0.832653,-1.102051,-1.176181,2.085306,-1.121726,3.224186,-1.281101,-1.340736,-1.165161,-1.174549,-1.320831,-0.963460,-1.186070,-1.004595,-1.303650,2.024994,-1.168787,-1.247598,-1.229762,-1.181903,1.037983,-0.832998,-0.529270,0.321560
2,-0.528387,-1.134755,-1.056973,0.950950,-0.029304,2.967266,2.363688,-1.269510,-1.100845,-1.033805,-1.288254,-1.253856,-1.170123,-1.318315,3.836658,-0.330308,1.445221,-1.224294,-1.018461,-1.343340,-1.223254,-0.818053,-1.095535,-1.086939,0.828890,-1.222408,-1.320656,-1.290488,-0.733490,-1.171299,-1.254606,-1.202827,-1.274763,-1.254401,-1.219053,-1.219178,-1.231398,3.139628,-1.175587,-1.253688,-1.238027,-1.253026,-1.197384,3.567114,-1.288216,-1.205349,-1.100337,-1.126014,2.055734,-1.238629,-1.216481,-1.198977,-1.406543,-1.352523,-0.855738,-0.639908,-1.264076,-0.644374,-1.293859,-1.108672,-1.142518,-1.160543,-1.196231,-1.262331,-0.025017,-1.123680,-1.071150,-1.138740,-0.133985,-0.348187,-1.201268,-1.250118,-1.285383,0.770705,-1.235573,2.041813,-1.155835,-0.208487,-1.249295,-1.123743,-0.385349,

In [28]:
pd.DataFrame(private_test_target.X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,8.875269,1.072714,0.349734,2.983766,3.551555,10.595364,17.327169,-0.144145,1.184181,3.041042,-0.429325,1.182045,0.099277,0.779827,8.188025,-0.270990,8.430601,1.836533,0.745340,-0.351743,0.439735,0.765868,1.125316,1.748645,19.122791,-0.242279,0.408039,2.850391,1.086136,-1.090475,0.438324,0.488942,-0.207521,0.550487,1.475966,1.382768,0.295792,8.440279,-0.722296,-0.775979,0.062518,0.936198,0.994711,18.807247,0.172296,0.722737,0.910600,0.162670,6.142643,0.404690,0.376773,1.482086,5.648309,-0.472074,0.724059,1.284304,-0.434013,0.619052,0.485545,3.045351,-0.635869,-0.436930,0.655264,0.467037,0.579335,-0.508544,0.959657,-0.067105,1.270376,0.234640,0.066366,0.192507,-0.682262,17.483824,-0.892266,10.238394,1.745744,2.714039,-0.701813,0.747299,0.877073,1.563599,1.058956,-0.457277,1.228947,0.388389,-0.010542,0.267340,5.669645,7.474632,1.808860,-0.199814,1.059669,-1.377614,0.698028,2.975605,0.312312,7.067345,0.448233,3.165192,1.711047,-0.512108,13.989281,-0.319555,1.991211,1.292144,8.198327,-0.497042,1.667113,0.175193,0.952262,2.701510,0.315933,1.213420,-0.912409,-0.586749,3.214163,-0.050946,0.610855,9.844093,-1.327427,6.832107,0.191609,-0.512926,0.011371,-0.843791,0.625344,1.954184,0.247891,1.342433,0.191373,11.086969,0.300469,0.805080,-0.291616,1.635413,7.509739,6.417010,-0.497571,3.759374
1,-0.331542,0.871559,0.484194,-0.784014,4.173365,5.242312,3.511576,1.208851,0.547345,-1.133742,1.118824,1.442501,0.455243,0.283438,6.993079,-0.026130,10.230215,2.101537,3.080711,-0.216307,0.533936,-0.498301,2.256030,0.834251,9.049059,0.779667,0.682752,0.573576,0.962821,0.293053,-0.178916,0.821061,0.655331,0.348021,-0.402928,0.473780,1.412509,16.912354,0.567827,-0.521395,-0.492781,0.638112,0.850088,-0.706945,0.518396,0.035253,1.234597,-0.144053,5.296485,0.349541,-0.509136,0.525733,0.911987,-0.256126,1.823956,1.146885,1.171728,3.219068,-0.451023,-0.466942,-0.415797,0.597951,-0.374317,1.043082,-0.423951,-0.193910,-0.043685,-0.116682,1.089144,1.671674,-0.377057,0.667307,0.713002,5.564362,-0.546455,9.313555,0.059604,1.048921,-0.461957,-0.665786,2.390503,-0.422717,-1.041684,1.245956,0.860920,1.365275,1.652736,1.011294,1.224288,0.010108,0.349052,-0.914434,0.693691,-0.017469,2.352008,-0.068106,0.663223,4.722332,-0.148873,3.343668,3.256628,-0.358878,2.278013,-1.183820,-0.309653,-0.683214,5.749076,1.312526,5.519828,0.472522,2.021698,2.179903,-0.207124,0.175539,-0.611545,2.168428,1.053179,0.262963,0.815993,-0.660271,1.183417,9.385386,2.176677,-0.303985,0.339248,-0.497396,0.404140,0.547107,0.947232,0.477131,0.310369,7.197814,-0.425260,-0.154081,-0.124246,-0.106671,3.812726,-0.155547,1.029169,3.598739
2,-0.211242,0.736260,1.096481,1.892444,1.585475,3.669683,2.232614,-1.042255,0.724554,0.833527,0.654052,1.568456,1.693014,1.225370,2.855914,0.092283,0.469575,1.260003,2.548044,-0.150811,0.455052,1.280808,-0.480674,0.291739,2.320207,0.187424,1.809985,0.079923,1.468113,1.157402,0.739877,-0.197212,-0.863157,1.058822,-0.267103,-0.118149,-0.085934,5.851235,-0.486316,-0.398280,0.354495,-0.619301,-0.555109,3.469603,-0.162294,-0.094670,1.391279,0.013668,-0.517603,0.099101,0.738166,1.355466,-0.590801,1.464079,6.004411,0.804768,0.427432,1.580613,0.833697,0.736071,2.144272,1.333905,-0.193387,1.040795,10.899550,1.741197,0.114915,0.051703,2.828831,0.678737,0.498896,0.860332,-0.265413,2.195580,0.749844,2.715751,0.203019,2.059091,0.831223,1.163601,1.132976,0.591543,0.990607,0.081153,2.316111,1.534367,0.996560,2.787698,0.651980,0.108140,0.825298,1.685943,-0.054280,0.446149,1.579761,0.015828,1.739001,6.939562,-1.162202,4.748300,0.575198,-0.

In [29]:
# pd.DataFrame(private_test_target_raw.X.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,10.0,8.0,11.0,7.0,30.0,15.0,21.0,2.0,23.0,9.0,1.0,1.0,4.0,2.0,6.0,0.0,14.0,9.0,2.0,0.0,12.0,8.0,2.0,4.0,23.0,1.0,1.0,7.0,17.0,11.0,10.0,3.0,2.0,3.0,2.0,9.0,4.0,6.0,4.0,0.0,4.0,3.0,5.0,60.0,4.0,10.0,2.0,2.0,6.0,5.0,14.0,9.0,11.0,0.0,5.0,22.0,4.0,3.0,1.0,4.0,0.0,28.0,12.0,12.0,1.0,1.0,3.0,4.0,9.0,2.0,39.0,36.0,0.0,23.0,5.0,20.0,4.0,9.0,0.0,2.0,1.0,3.0,13.0,2.0,3.0,2.0,8.0,4.0,28.0,21.0,13.0,11.0,46.0,3.0,5.0,1.0,3.0,25.0,7.0,7.0,11.0,0.0,7.0,2.0,6.0,3.0,13.0,0.0,27.0,1.0,10.0,8.0,4.0,1.0,0.0,0.0,9.0,1.0,1.0,16.0,0.0,18.0,1.0,0.0,4.0,1.0,3.0,9.0,5.0,9.0,13.0,22.0,3.0,36.0,0.0,11.0,48.0,39.0,5.0,17.0
1,0.0,6.0,10.0,0.0,34.0,6.0,3.0,4.0,16.0,0.0,3.0,1.0,4.0,1.0,5.0,0.0,18.0,9.0,5.0,0.0,11.0,3.0,3.0,2.0,8.0,2.0,1.0,2.0,14.0,17.0,6.0,3.0,3.0,2.0,0.0,5.0,6.0,15.0,7.0,0.0,2.0,2.0,4.0,0.0,4.0,6.0,2.0,1.0,5.0,4.0,8.0,5.0,2.0,0.0,7.0,19.0,8.0,8.0,0.0,0.0,0.0,35.0,6.0,13.0,0.0,1.0,1.0,3.0,7.0,4.0,29.0,37.0,1.0,5.0,5.0,17.0,1.0,4.0,0.0,0.0,2.0,0.0,3.0,5.0,2.0,3.0,14.0,5.0,6.0,1.0,6.0,6.0,36.0,6.0,9.0,0.0,3.0,14.0,4.0,7.0,17.0,0.0,1.0,0.0,1.0,0.0,8.0,1.0,153.0,1.0,13.0,6.0,2.0,0.0,0.0,2.0,3.0,1.0,1.0,0.0,4.0,28.0,3.0,0.0,4.0,1.0,2.0,4.0,6.0,5.0,12.0,12.0,1.0,22.0,0.0,4.0,18.0,3.0,9.0,15.0
2,0.0,5.0,12.0,4.0,15.0,4.0,2.0,0.0,16.0,3.0,2.0,1.0,7.0,2.0,2.0,0.0,1.0,6.0,4.0,0.0,10.0,8.0,0.0,1.0,2.0,1.0,2.0,1.0,16.0,22.0,9.0,1.0,0.0,3.0,0.0,3.0,2.0,4.0,3.0,0.0,4.0,0.0,1.0,5.0,2.0,5.0,2.0,1.0,0.0,3.0,13.0,7.0,0.0,1.0,24.0,16.0,5.0,4.0,1.0,1.0,2.0,42.0,6.0,12.0,14.0,4.0,1.0,3.0,13.0,2.0,37.0,37.0,0.0,2.0,9.0,4.0,1.0,6.0,1.0,2.0,1.0,1.0,10.0,2.0,4.0,3.0,10.0,10.0,4.0,1.0,7.0,19.0,26.0,7.0,6.0,0.0,5.0,23.0,1.0,10.0,5.0,0.0,2.0,4.0,2.0,0.0,6.0,0.0,152.0,20.0,8.0,36.0,0.0,0.0,2.0,0.0,4.0,1.0,0.0,11.0,2.0,6.0,0.0,0.0,2.0,3.0,2.0,8.0,6.0,2.0,9.0,15.0,5.0,29.0,0.0,6.0,7.0,5.0,7.0,8.0
3,4.0,8.0,11.0,5.0,40.0,10.0,13.0,4.0,29.0,2.0,5.0,0.0,5.0,1.0,7.0,0.0,3.0,6.0,3.0,0.0,18.0,7.0,1.0,5.0,9.0,3.0,1.0,2.0,13.0,24.0,6.0,4.0,2.0,3.0,2.0,6.0,6.0,2.0,8.0,0.0,3.0,2.0,5.0,24.0,6.0,9.0,1.0,2.0,3.0,3.0,12.0,3.0,3.0,0.0,6.0,14.0,5.0,6.0,1.0,0.0,3.0,41.0,15.0,11.0,3.0,4.0,0.0,4.0,12.0,2.0,49.0,37.0,1.0,28.0,7.0,0.0,1.0,4.0,0.0,1.0,3.0,4.0,12.0,3.0,1.0,2.0,10.0,3.0,13.0,2.0,5.0,9.0,49.0,5.0,9.0,0.0,2.0,28.0,4.0,7.0,6.0,0.0,4.0,3.0,2.0,0.0,5.0,0.0,47.0,14.0,10.0,8.0,7.0,1.0,1.0,0.0,4.0,1.0,0.0,11.0,1.0,7.0,1.0,0.0,3.0,1.0,0.0,5.0,3.0,7.0,11.0,10.0,5.0,34.0,0.0,6.0,39.0,13.0,10.0,15.0
4,0.0,3.0,10.0,1.0,14.0,3.0,3.0,2.0,11.0,2.0,0.0,0.0,2.0,0.0,3.0,0.0,1.0,2.0,2.0,0.0,4.0,5.0,1.0,1.0,4.0,0.0,0.0,0.0,3.0,9.0,2.0,1.0,3.0,1.0,0.0,2.0,3.0,1.0,4.0,1.0,3.0,2.0,2.0,3.0,5.0,11.0,0.0,4.0,1.0,0.0,5.0,2.0,0.0,1.0,3.0,11.0,2.0,1.0,0.0,0.0,0.0,27.0,7.0,4.0,0.0,1.0,2.0,1.0,7.0,2.0,22.0,17.0,1.0,0.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,7.0,4.0,23.0,3.0,6.0,0.0,0.0,15.0,2.0,2.0,7.0,0.0,0.0,3.0,14.0,0.0,2.0,0.0,34.0,1.0,5.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,7.0,3.0,2.0,7.0,3.0,1.0,15.0,0.0,6.0,8.0,1.0,6.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26862,0.0,6.0,4.0,11.0,31.0

In [30]:
pd.concat([pd.DataFrame(pred_private), pd.DataFrame(private_test_target.X)]).corr().head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,1.000000,0.395669,0.483408,0.488264,0.477773,0.429052,0.566816,0.323234,0.464535,0.501602,0.482766,0.408244,0.477250,0.441145,0.520438,0.446120,0.429431,0.413282,0.402171,0.337965,0.441410,0.141506,0.415503,0.499274,0.459079,0.493078,0.463867,0.498512,0.495758,0.497030,0.490012,0.429719,0.483207,0.476248,0.438478,0.478820,0.433683,0.260158,0.452913,0.416330,0.435619,0.438191,0.484492,0.404008,0.491952,0.492421,0.359967,0.379979,0.301469,0.488284,0.459609,0.478537,0.455014,0.449035,0.408021,0.478971,0.470944,0.427142,0.446146,0.472947,0.490029,0.526877,0.460415,0.489742,0.412616,0.413162,0.348802,0.442170,0.393929,0.483109,0.519561,0.521708,0.436926,0.475813,0.488602,0.292039,0.442170,0.374262,0.431792,0.241839,0.484195,0.209908,0.533824,0.443940,0.482815,0.488575,0.515723,0.497851,0.390620,0.022936,0.422553,0.516082,0.518356,0.437861,0.481706,0.302802,0.346569,0.495138,0.485233,0.277826,0.484969,0.431210,0.401646,0.423084,0.284657,0.464332,0.520663,0.425381,0.231320,0.428299,0.334862,0.416166,0.484221,0.045695,0.526755,0.359470,0.383579,0.478113,0.466129,0.483809,0.462295,0.165481,0.446337,0.404827,0.450653,0.447470,0.476096,0.528573,0.486120,0.464129,0.484039,0.474103,0.417733,0.519616,0.409648,0.472714,0.407666,0.550892,0.200721,0.393039
1,0.395669,1.000000,0.746622,0.575627,0.723880,0.667003,0.172938,0.741555,0.733831,0.595802,0.638539,0.578936,0.658260,0.630484,0.280495,0.228621,0.598998,0.332825,0.484136,0.487902,0.668161,0.716688,0.635515,0.635909,0.278187,0.609422,0.629152,0.639810,0.642756,0.717656,0.717845,0.621513,0.668676,0.646064,0.602503,0.676333,0.664232,0.630006,0.718724,0.637160,0.629878,0.629554,0.648044,0.532151,0.640098,0.713152,0.566501,0.759951,0.176878,0.677851,0.695723,0.687360,0.566555,0.590644,0.680238,0.677953,0.689352,0.452046,0.583545,0.591108,0.489226,0.700364,0.708384,0.720770,0.371190,0.611794,0.573622,0.672325,0.681164,0.427318,0.733042,0.729567,0.623505,0.207042,0.672603,0.432422,0.652245,0.404376,0.625941,0.737245,0.238030,0.714350,0.699148,0.690668,0.629014,0.632924,0.693314,0.683341,0.570113,0.580825,0.670754,0.706894,0.751393,0.707699,0.551552,0.481998,0.782548,0.737967,0.688871,0.617967,0.618359,0.586544,0.208264,0.650952,0.355633,0.618192,0.544771,0.587298,0.424412,0.331822,0.518739,0.670601,0.465140,0.520835,0.554927,0.318087,0.518064,0.622578,0.468386,0.380995,0.676549,0.567254,0.630332,0.591158,0.657922,0.663050,0.652000,0.584708,0.695618,0.747702,0.670902,0.452958,0.624643,0.716503,0.338747,0.735621,0.406887,0.477481,0.414640,0.658691
2,0.483408,0.746622,1.000000,0.621547,0.693663,0.596770,0.287629,0.671602,0.815657,0.668238,0.690057,0.651008,0.730867,0.703806,0.428624,0.322205,0.568021,0.406425,0.546555,0.537525,0.756375,0.582961,0.657365,0.686821,0.417182,0.679181,0.683047,0.693969,0.736968,0.803143,0.787063,0.707430,0.725986,0.711906,0.662414,0.736273,0.755528,0.483741,0.792542,0.689833,0.699645,0.714330,0.726879,0.478732,0.694275,0.771409,0.671429,0.665529,0.255545,0.746812,0.793852,0.760781,0.616719,0.646954,0.748566,0.715564,0.767617,0.546022,0.650030,0.658417,0.562305,0.789693,0.795482,0.798212,0.447375,0.710573,0.684138,0.751747,0.744213,0.509072,0.808306,0.804600,0.685271,0.335996,0.738999,0.517030,0.722997,0.541703,0.675274,0.563407,0.339023,0.515474,0.768457,0.753326,0.684555,0.698054,0.782506,0.741403,0.631268,0.356916,0.771867,0.776911,0.827655,0.787617,0.581479,0.478580,0.708955,0.731530,0.759596,0.494265,0.638468,0.644143,0.313197,0.739695,0.457568,0.699670,0.586008,0.655927,0.571842,0.454210,0.668346,0.653507,0.571339,0.3566